# 20: Extend Dataset

In this notebook, new and initally not planned features are added to the data

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from datetime import datetime
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import shapely
import hashlib
import holidays
from shapely import Point, wkt
import scipy.stats
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
data_month_double = '07'

## Load data

In [3]:
# read final data
data = pd.read_csv("../Data/18_Merged_with_Weather.csv")

/var/folders/_m/71fq0nk106s02qxvct5s2sv80000gn/T/ipykernel_2095/3797065100.py:2: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("/Volumes/T7/Master/Processed Data/"+data_month_double+"/18_Merged_with_Weather_NEU_NEU_TEST.csv")


In [4]:
tier = pd.read_parquet("../Data/12_part2.parquet")
next = pd.read_parquet("../Data/15_part2.parquet")

In [5]:
tier['trip_duration_seconds'] = tier['trip_duration'].dt.total_seconds()
next['trip_duration_seconds'] = next['trip_duration'].dt.total_seconds()

In [6]:
# transform geometry
data['station_point'] = data['station_point'].apply(wkt.loads)

In [7]:
data['buffer_zone'] = shapely.wkt.loads(data['buffer_zone'])

In [8]:
data = gpd.GeoDataFrame(data, geometry='station_point')

In [9]:
data = data.loc[:, ~data.columns.str.contains('Unnamed')]

In [10]:
#data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.2'], inplace=True)

In [11]:
data['actual_arrival_time'] = pd.to_datetime(data['actual_arrival_time'])
data['actual_departure_time'] = pd.to_datetime(data['actual_departure_time'])
data['scheduled_arrival_time'] = pd.to_datetime(data['scheduled_arrival_time'])
data['scheduled_departure_time'] = pd.to_datetime(data['scheduled_departure_time'])

In [12]:
data = gpd.GeoDataFrame(data, geometry='buffer_zone')

In [13]:
data['date'] = pd.to_datetime(data['date'])

In [14]:
data['arrival_delay'] = pd.to_timedelta(data['arrival_delay'])
data['departure_delay'] = pd.to_timedelta(data['departure_delay'])

## Add features

In [15]:
data['arrival_delay_float'] = data['arrival_delay'].astype(str)
data['arrival_delay_float'] = data['arrival_delay_float'].str.split(' ', n=2).str[-1]
data['arrival_delay_float'] = data['arrival_delay_float'].astype(str)
data['arrival_delay_float'] = pd.to_timedelta(data['arrival_delay_float'])
data['arrival_delay_float'] = pd.to_timedelta(data['arrival_delay_float']).dt.total_seconds() / 60

In [16]:
data['departure_delay_float'] = data['departure_delay'].astype(str)
data['departure_delay_float'] = data['departure_delay_float'].str.split(' ', n=2).str[-1]
data['departure_delay_float'] = data['departure_delay_float'].astype(str)
data['departure_delay_float'] = pd.to_timedelta(data['departure_delay_float'])
data['departure_delay_float'] = pd.to_timedelta(data['departure_delay_float']).dt.total_seconds() / 60

In [17]:
data.columns

Index(['route_id', 'agency_id', 'route_short_name', 'route_type',
       'route_type_name', 'agency_name', 'service_id', 'trip_id',
       'trip_headsign', 'direction_id', 'shape_id', 'stop_id',
       'actual_arrival_time', 'actual_departure_time', 'vrs_timestamp',
       'stop_sequence', 'stop_headsign', 'stop_name', 'scheduled_arrival_time',
       'scheduled_departure_time', 'arrival_delay', 'departure_delay',
       'service', 'date', 'weekday', 'transfer_stop', 'tier_trips_count',
       'tier_trips_id', 'tier_trips_end_at_station_count',
       'tier_trips_end_at_station_id', 'station_point', 'buffer_zone',
       'nextbike_trips_count', 'nextbike_trips_id',
       'nextbike_trips_end_at_station_count',
       'nextbike_trips_end_at_station_id', 'current_time', 'current_temp',
       'current_feels_like', 'current_cloudiness', 'current_visibility',
       'current_wind_speed', 'current_description',
       'current_precipitation_volume', 'arrival_delay_float',
       'departure_

In [18]:
data['actual_arrival_time_float'] = data['actual_arrival_time'].dt.time.astype(str).str.replace(':', '').replace('NaT', '0').astype(int)
data['actual_departure_time_float'] = data['actual_departure_time'].dt.time.astype(str).str.replace(':', '').replace('NaT', '0').astype(int)

In [19]:
data['actual_arrival_time']

0                         NaT
1                         NaT
2                         NaT
3                         NaT
4                         NaT
                  ...        
2564856                   NaT
2564857                   NaT
2564858                   NaT
2564859   2023-08-01 03:24:00
2564860                   NaT
Name: actual_arrival_time, Length: 2564861, dtype: datetime64[ns]

In [20]:
data['actual_arrival_time_float']

0              0
1              0
2              0
3              0
4              0
           ...  
2564856        0
2564857        0
2564858        0
2564859    32400
2564860        0
Name: actual_arrival_time_float, Length: 2564861, dtype: int64

In [21]:
# create column delay span


In [22]:
data['delay_span'] = '0'

for index in data.index:
    if (data['departure_delay'][index] > pd.Timedelta(0, unit='m')) and (data['departure_delay'][index] <= pd.Timedelta(5, unit='m')):
        data['delay_span'][index] = '0-5'
    elif (data['departure_delay'][index] > pd.Timedelta(5, unit='m')) and (data['departure_delay'][index] <= pd.Timedelta(10, unit='m')):
        data['delay_span'][index] = '5-10'
    elif (data['departure_delay'][index] > pd.Timedelta(10, unit='m')) and (data['departure_delay'][index] <= pd.Timedelta(15, unit='m')):
        data['delay_span'][index] = '10-15'
    elif (data['departure_delay'][index] > pd.Timedelta(15, unit='m')) and (data['departure_delay'][index] <= pd.Timedelta(20, unit='m')):
        data['delay_span'][index] = '15-20'
    elif (data['departure_delay'][index] > pd.Timedelta(20, unit='m')) and (data['departure_delay'][index] <= pd.Timedelta(30, unit='m')):
        data['delay_span'][index] = '20-30'
    elif (data['departure_delay'][index] > pd.Timedelta(30, unit='m')) and (data['departure_delay'][index] <= pd.Timedelta(45, unit='m')):
        data['delay_span'][index] = '30-45'
    elif (data['departure_delay'][index] > pd.Timedelta(45, unit='m')) and (data['departure_delay'][index] <= pd.Timedelta(60, unit='m')):
        data['delay_span'][index] = '45-60'

/var/folders/_m/71fq0nk106s02qxvct5s2sv80000gn/T/ipykernel_2095/1819807835.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data['delay_span'][index] = '0-5'
/var/folders/_m/71fq0nk106s02qxvct5s2sv80000gn/T/ipykernel_2095/1819807835.py:5:

In [23]:
data['arrival_delay_span'] = '0'

for index in data.index:
    if (data['arrival_delay'][index] > pd.Timedelta(0, unit='m')) and (data['arrival_delay'][index] <= pd.Timedelta(5, unit='m')):
        data['delay_span'][index] = '0-5'
    elif (data['arrival_delay'][index] > pd.Timedelta(5, unit='m')) and (data['arrival_delay'][index] <= pd.Timedelta(10, unit='m')):
        data['delay_span'][index] = '5-10'
    elif (data['arrival_delay'][index] > pd.Timedelta(10, unit='m')) and (data['arrival_delay'][index] <= pd.Timedelta(15, unit='m')):
        data['delay_span'][index] = '10-15'
    elif (data['arrival_delay'][index] > pd.Timedelta(15, unit='m')) and (data['arrival_delay'][index] <= pd.Timedelta(20, unit='m')):
        data['delay_span'][index] = '15-20'
    elif (data['arrival_delay'][index] > pd.Timedelta(20, unit='m')) and (data['arrival_delay'][index] <= pd.Timedelta(30, unit='m')):
        data['delay_span'][index] = '20-30'
    elif (data['arrival_delay'][index] > pd.Timedelta(30, unit='m')) and (data['arrival_delay'][index] <= pd.Timedelta(45, unit='m')):
        data['delay_span'][index] = '30-45'
    elif (data['arrival_delay'][index] > pd.Timedelta(45, unit='m')) and (data['arrival_delay'][index] <= pd.Timedelta(60, unit='m')):
        data['delay_span'][index] = '45-60'

/var/folders/_m/71fq0nk106s02qxvct5s2sv80000gn/T/ipykernel_2095/633990372.py:17: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data['delay_span'][index] = '45-60'
/var/folders/_m/71fq0nk106s02qxvct5s2sv80000gn/T/ipykernel_2095/633990372.py:1

In [24]:
data.columns

Index(['route_id', 'agency_id', 'route_short_name', 'route_type',
       'route_type_name', 'agency_name', 'service_id', 'trip_id',
       'trip_headsign', 'direction_id', 'shape_id', 'stop_id',
       'actual_arrival_time', 'actual_departure_time', 'vrs_timestamp',
       'stop_sequence', 'stop_headsign', 'stop_name', 'scheduled_arrival_time',
       'scheduled_departure_time', 'arrival_delay', 'departure_delay',
       'service', 'date', 'weekday', 'transfer_stop', 'tier_trips_count',
       'tier_trips_id', 'tier_trips_end_at_station_count',
       'tier_trips_end_at_station_id', 'station_point', 'buffer_zone',
       'nextbike_trips_count', 'nextbike_trips_id',
       'nextbike_trips_end_at_station_count',
       'nextbike_trips_end_at_station_id', 'current_time', 'current_temp',
       'current_feels_like', 'current_cloudiness', 'current_visibility',
       'current_wind_speed', 'current_description',
       'current_precipitation_volume', 'arrival_delay_float',
       'departure_

In [25]:
data['weather_int'] = 0

for index in data.index:
    if data['current_description'][index] == 'clear sky' or data['current_description'][index] == 'few clouds' or data['current_description'][index] == 'scattered clouds' or data['current_description'][index] == 'broken clouds' or data['current_description'][index] == 'overcast clouds':
        data['weather_int'][index] = 0 # DRY
    elif data['current_description'][index] == 'light rain' or data['current_description'][index] == 'light intensity drizzle' or data['current_description'][index] == 'drizzle' or data['current_description'][index] == 'light intensity drizzle rain' or data['current_description'][index] == 'light intensity shower rain':
        data['weather_int'][index] = 1 # light rain
    elif data['current_description'][index] == 'moderate rain' or data['current_description'][index] == 'heavy intensity rain' or data['current_description'][index] == 'very heavy rain' or data['current_description'][index] == 'thunderstorm with rain' or data['current_description'][index] == 'thunderstorm with heavy rain' or data['current_description'][index] == 'shower rain' or data['current_description'][index] == 'heavy intensity shower rain':
        data['weather_int'][index] = 2 # rain

/var/folders/_m/71fq0nk106s02qxvct5s2sv80000gn/T/ipykernel_2095/1550837342.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data['weather_int'][index] = 0 # DRY
/var/folders/_m/71fq0nk106s02qxvct5s2sv80000gn/T/ipykernel_2095/1550837342.py

In [30]:
# create column timespan

data['time_span_arrival'] = '0'
data['time_span_int_arrival'] = 0

for index in data.index:
    if (data['actual_arrival_time'][index].hour >= 6) & (data['actual_arrival_time'][index].hour <= 9):
        data['time_span_arrival'][index] = 'morning rush'
        data['time_span_int_arrival'][index] = 1
    elif (data['actual_arrival_time'][index].hour > 9) & (data['actual_arrival_time'][index].hour < 12):
        data['time_span_arrival'][index] = 'until midday'
        data['time_span_int_arrival'][index] = 2
    elif (data['actual_arrival_time'][index].hour >= 12) & (data['actual_arrival_time'][index].hour <= 14):
        data['time_span_arrival'][index] = 'lunch'
        data['time_span_int_arrival'][index] = 3
    elif (data['actual_arrival_time'][index].hour > 14) & (data['actual_arrival_time'][index].hour < 17):
        data['time_span_arrival'][index] = 'afternoon'
        data['time_span_int_arrival'][index] = 4
    elif (data['actual_arrival_time'][index].hour >= 17) & (data['actual_arrival_time'][index].hour <= 19):
        data['time_span_arrival'][index] = 'evening rush'
        data['time_span_int_arrival'][index] = 5
    elif (data['actual_arrival_time'][index].hour > 19) & (data['actual_arrival_time'][index].hour < 22):
        data['time_span_arrival'][index] = 'evening'
        data['time_span_int_arrival'][index] = 6
    elif (data['actual_arrival_time'][index].hour >= 22) & (data['actual_arrival_time'][index].hour <= 2):
        data['time_span_arrival'][index] = 'night'
        data['time_span_int_arrival'][index] = 7
    elif (data['actual_arrival_time'][index].hour > 2) & (data['actual_arrival_time'][index].hour < 6):
        data['time_span_arrival'][index] = 'early morning'
        data['time_span_int_arrival'][index] = 8

/var/folders/_m/71fq0nk106s02qxvct5s2sv80000gn/T/ipykernel_2095/1889924452.py:29: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data['time_span_arrival'][index] = 'early morning'
/var/folders/_m/71fq0nk106s02qxvct5s2sv80000gn/T/ipykernel_209

In [32]:
# create column timespan

data['time_span'] = '0'
data['time_span_int'] = 0

for index in data.index:
    if (data['actual_departure_time'][index].hour >= 6) & (data['actual_departure_time'][index].hour <= 9):
        data['time_span'][index] = 'morning rush'
        data['time_span_int'][index] = 1
    elif (data['actual_departure_time'][index].hour > 9) & (data['actual_departure_time'][index].hour < 12):
        data['time_span'][index] = 'until midday'
        data['time_span_int'][index] = 2
    elif (data['actual_departure_time'][index].hour >= 12) & (data['actual_departure_time'][index].hour <= 14):
        data['time_span'][index] = 'lunch'
        data['time_span_int'][index] = 3
    elif (data['actual_departure_time'][index].hour > 14) & (data['actual_departure_time'][index].hour < 17):
        data['time_span'][index] = 'afternoon'
        data['time_span_int'][index] = 4
    elif (data['actual_departure_time'][index].hour >= 17) & (data['actual_departure_time'][index].hour <= 19):
        data['time_span'][index] = 'evening rush'
        data['time_span_int'][index] = 5
    elif (data['actual_departure_time'][index].hour > 19) & (data['actual_departure_time'][index].hour < 22):
        data['time_span'][index] = 'evening'
        data['time_span_int'][index] = 6
    elif (data['actual_departure_time'][index].hour >= 22) & (data['actual_departure_time'][index].hour <= 2):
        data['time_span'][index] = 'night'
        data['time_span_int'][index] = 7
    elif (data['actual_departure_time'][index].hour > 2) & (data['actual_departure_time'][index].hour < 6):
        data['time_span'][index] = 'early morning'
        data['time_span_int'][index] = 8

/var/folders/_m/71fq0nk106s02qxvct5s2sv80000gn/T/ipykernel_2095/837669781.py:29: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data['time_span'][index] = 'early morning'
/var/folders/_m/71fq0nk106s02qxvct5s2sv80000gn/T/ipykernel_2095/8376697

In [33]:
data['weekend'] = 0

for index in data.index:
    if data['weekday'][index] == 'Monday' or data['weekday'][index] == 'Tuesday' or data['weekday'][index] == 'Wednesday' or data['weekday'][index] == 'Thursday':
        data['weekend'][index] = 0
    if data['weekday'][index] == 'Friday':
        data['weekend'][index] = 1
    if data['weekday'][index] == 'Saturday':
        data['weekend'][index] = 2
    if data['weekday'][index] == 'Sunday':
        data['weekend'][index] = 3

/var/folders/_m/71fq0nk106s02qxvct5s2sv80000gn/T/ipykernel_2095/293593914.py:7: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data['weekend'][index] = 1
/var/folders/_m/71fq0nk106s02qxvct5s2sv80000gn/T/ipykernel_2095/293593914.py:7: SettingW

In [34]:
data.to_csv("/Volumes/T7/Master/Processed Data/"+data_month_double+"/20_Final_Data.csv")